# ProtSpace: Interactive Protein Embedding Visualization
### About ProtSpace ([github](https://github.com/tsenoner/protspace))
ProtSpace is a tool for interactive visualization of protein embeddings that:
- Converts high-dimensional protein embeddings into 2D/3D visualizations
- Supports multiple dimension reduction methods (PCA, UMAP, t-SNE, PaCMAP)
- Allows annotation-based coloring and shaping of data points
- Integrates protein structure visualization alongside embedding space
- Enables publication-quality exports and sharing of visualization sessions

In [ ]:
#@title Install Dependencies + Import Libraries + Download example files (~2min)
#@markdown This will install ProtSpace and download pre-generated JSON example files
%%capture
import sys
import os
from pathlib import Path
try:
    from protspace.app import ProtSpace
except ImportError:
    # Only run installation if not already installed
    !pip install -q protspace
    from protspace.app import ProtSpace

# Download example files if they don't exist
if not Path('protspace').exists():
    !wget -q -O protspace.zip -r --no-parent -nH --cut-dirs=3 --reject "index.html*" -e robots=off https://nextcloud.in.tum.de/index.php/s/jb9fN3wawgTRswS/download
    !unzip -q -j protspace.zip -d protspace

In [ ]:
#@title 📂 Select JSON File {display-mode: "form"}
#@markdown ### Choose a JSON file from the directory

from ipywidgets import Dropdown
from IPython.display import display
from pathlib import Path

class FileSelector:
    def __init__(self, path='protspace'):
        self.file_path = Path(path)
        if not self.file_path.exists():
            print(f"⚠️ Directory '{path}' not found. Please run the installation cell first.")
            return
        self.setup_selector()

    def setup_selector(self):
        files = sorted(f.name for f in self.file_path.glob('*.json'))
        if not files:
            print("⚠️ No JSON files found in the directory.")
            return

        self.selected_file = files[0]
        self.dropdown = Dropdown(
            options=files,
            description='JSON Files:',
            value=self.selected_file,
            style={'description_width': 'auto'},
            # layout={'width': 'auto'}
        )
        self.dropdown.observe(self._on_change, names='value')
        display(self.dropdown)
        print(f"💡 Found {len(files)} JSON files")

    def _on_change(self, change):
        self.selected_file = change.new

    def get_selected_filepath(self):
        return str(self.file_path / self.selected_file) if hasattr(self, 'selected_file') else None

selector = FileSelector()

### Navigation Guide
- interact with the 2D plot:
  - **Select & Zoom**: Click and hold left mouse button to select an area and zoom into
  - **Reset View**: Double-click on the plot to return to full visualization
- interact with 3D plot:
  - **Orbital Rotation**: Click and hold the left mouse button
  - **Pan**: Click and hold the right mouse button
  - **Zoom**: Use mouse wheel while cursor is in the graph
- common interactions:
  - **Legend Interaction**:
    - Click on a group to hide/show it
    - Double-click on a displayed group to isolate it (double-click again to show all groups)
  - **Hover Info**: Mouse over points to see detailed information
  - **Molecule Selection**: Click on individual data points to select molecules (if PDB file is provided)
  - **Settings**: Costumize the dot color and shapes the gear icon
  - **Download**: Save the view as HTML or PNG using the download button

In [ ]:
#@title 🌟 Visualization Settings and Launch {display-mode: "form"}
#@markdown ### Configure and launch ProtSpace visualization

#@markdown #### Visualization Height (pixels):
jupyter_height = 800 #@param {type:"slider", min:400, max:1200, step:50}

#@markdown #### Jupyter Display Mode:
jupyter_mode = "inline" #@param ["inline", "external"]

import os
import sys
from IPython.display import display, HTML, clear_output

def show_launch_info(json_file, height, mode):
    """Display launch information in a formatted box separate from the dashboard."""
    info_html = f"""
    <div style="background-color: #666666; padding: 10px; margin-bottom: 20px; border-radius: 5px;">
        <p style="margin: 0;"><strong>📊 Launching ProtSpace with:</strong></p>
        <ul style="margin: 10px 0;">
            <li>File: {os.path.basename(json_file)}</li>
            <li>Height: {height}px</li>
            <li>Mode: {mode}</li>
        </ul>
    </div>
    """
    display(HTML(info_html))

def launch_protspace(json_file, height=800, mode="inline"):
    """
    Launch ProtSpace with customized visualization settings.

    Args:
        json_file: Path to the JSON configuration file
        height: Height of the visualization in pixels
        mode: Jupyter display mode ('inline', 'external', 'jupyterlab', 'tab')
    """
    if not os.path.exists(json_file):
        display(HTML(
            '<div style="color: #dc3545; padding: 10px;">'
            f'⚠️ JSON file not found: {json_file}'
            '</div>'
        ))
        return False

    # Clear any previous output
    clear_output(wait=True)

    # Show launch information
    show_launch_info(json_file, height, mode)

    original_stdout, original_stderr = sys.stdout, sys.stderr
    try:
        sys.stdout = sys.stderr = open(os.devnull, 'w')
        protspace = ProtSpace(default_json_file=json_file)
        app = protspace.create_app()
        app.run(
            jupyter_mode=mode,
            jupyter_height=height,  # For inline mode, apply height restriction
            jupyter_width='100%',
            dev_tools_silence_routes_logging=True,
            dev_tools_prune_errors=True
        )
        return True
    except Exception as e:
        display(HTML(
            '<div style="color: #dc3545; padding: 10px;">'
            f'❌ Error launching ProtSpace: {str(e)}'
            '</div>'
        ))
        return False
    finally:
        sys.stdout, sys.stderr = original_stdout, original_stderr

# Get the selected file path and launch
json_file = selector.get_selected_filepath()

if json_file:
    launch_protspace(json_file, height=jupyter_height, mode=jupyter_mode)
else:
    display(HTML(
        '<div style="color: #dc3545; padding: 10px;">'
        '⚠️ No file selected. Please choose a file from the dropdown above.'
        '</div>'
    ))